In [22]:
import pprint
import sklearn
import pandas as pd
import yaml
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


In [17]:
def get_hyperparameters(job_id):
    # Update file name with correct path
    with open("rf_hyperparams.yml", 'r') as stream:
        hyper_param_set = yaml.load(stream)
    print("\nHypermeter set for job_id: ",job_id)
    print("------------------------------------")
    pprint.pprint(hyper_param_set[job_id-1]["hyperparam_set"])
    print("------------------------------------\n")

    return hyper_param_set[job_id-1]["hyperparam_set"]


In [3]:
os.environ['JOB_ID']="1"

In [4]:
job_id = int(os.environ['JOB_ID'])
print(job_id)

1


In [9]:
#!wget http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data .
    

--2020-04-18 16:29:40--  http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2020-04-18 16:29:40 (11.4 MB/s) - ‘iris.data’ saved [4551/4551]

--2020-04-18 16:29:40--  http://./
Resolving . (.)... failed: Name or service not known.
wget: unable to resolve host address ‘.’
FINISHED --2020-04-18 16:29:40--
Total wall clock time: 0.4s
Downloaded: 1 files, 4.4K in 0s (11.4 MB/s)


In [11]:
job_id = int(os.environ['JOB_ID'])
DATA_DIR = '.' 
df = pd.read_csv(DATA_DIR+"/iris.data", sep=",")
df.columns = ['sepal length','sepal width','petal length','petal width', 'species']


In [13]:
df.head()

,sepal length,sepal width,petal length,petal width,species
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [19]:
X = df[['sepal length', 'sepal width', 'petal length', 'petal width']]  # Features
y = df['species']  # Labels

    # Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)  # 70% training and 30% test

    # Load hyperparameters
hyperparams = get_hyperparameters(job_id)
n_est = hyperparams["n_estim"]
max_depth = hyperparams['max_depth']


Hypermeter set for job_id:  1
------------------------------------
{'max_depth': 5, 'n_estim': 100}
------------------------------------



/home/aleks/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [20]:
clf = RandomForestClassifier(n_estimators=n_est, max_depth=max_depth)

    # Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [24]:
acc = metrics.accuracy_score(y_test, y_pred)
    #print("Accuracy:", metrics.accuracy_score(y_test, y_pred))



In [27]:
res_df = pd.DataFrame({'accuracy': acc}, index=[0])


In [29]:
file_name = './results_job_id_'+str(job_id)+'.csv'
res_df.to_csv(file_name, index=False)